## <span style='color: green;'>DESCRIBE</span> CO-occurrence Matrix<span style='color: red; float: right'>WORK IN PROGRESS</span>


In [ ]:
import gensim
#corpus = get_current_corpus().textacy_corpus

nlp = textacy.load_spacy('en')
text = "The Horse Raced Past the Barn Fell"

corpus = textacy.Corpus(nlp)
corpus.add_text(text)
    
term_args = dict(
    args=dict(
        #ngrams=gui.ngrams.value,
        #named_entities=gui.named_entities.value,
        normalize='lemma',
        as_strings=True
    ),
    kwargs=dict(
        filter_nums=True,
        drop_determiners=True,
        min_freq=1,
        include_pos=['NOUN'],
        filter_stops=True,
        filter_punct=True
    ),
    extra_stop_words=None
)
terms_iter = lambda: ( textacy_utility.textacy_filter_terms(doc, term_args) for doc in corpus )

#vocab = corpus.spacy_vocab
#id2token = { x.lower: x.lower_ for x in vocab }
#dictionary.token2id = { x.lower_: x.lower for x in vocab }
dictionary = gensim.corpora.Dictionary(terms_iter())
_ = dictionary[0]
X = gensim.topic_coherence.text_analysis.WordOccurrenceAccumulator(dictionary.id2token, dictionary)
X.accumulate(terms_iter(), window_size=2)

In [131]:
import gensim

nlp = textacy.load_spacy('en')
text = "The basic concept of the word association".lower()

corpus = textacy.Corpus(nlp)
corpus.add_text(text)
fx = lambda: ((x.orth_ for x in doc) for doc in corpus)
dictionary = gensim.corpora.Dictionary(fx())
_ = dictionary[0]
X = gensim.topic_coherence.text_analysis.WordOccurrenceAccumulator(dictionary.id2token, dictionary)
X.accumulate(fx(), window_size=2)
coo = X._co_occurrences.tocoo()
df = pd.DataFrame({'word_id1': coo.row, 'word_id2': coo.col, 'count': coo.data})
#df = df[df.word_id1 > df.word_id2]

df['word1'] = df.word_id1.apply(lambda x: dictionary[x])
df['word2'] = df.word_id2.apply(lambda x: dictionary[x])
df['count1'] = df.word_id1.apply(lambda x: X._occurrences[x])
df['count2'] = df.word_id2.apply(lambda x: X._occurrences[x])

df = df[['word_id1', 'word_id2', 'word1', 'word2', 'count', 'count1', 'count2']].sort_values(['word1'], ascending=True)

df

2018-12-14 14:59:45,961 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-12-14 14:59:45,962 : INFO : built Dictionary(6 unique tokens: ['of', 'the', 'concept', 'basic', 'word']...) from 1 documents (total 7 corpus positions)


In [ ]:
import itertools

def corpus_subset(corpus, px):
    iter1, iter2 = itertools.tee(corpus.get(px))
    docs = (d.spacy_doc for d in iter1)
    metadatas = (d.metadata for d in iter2)
    sub_corpus = textacy.Corpus(lang=corpus.spacy_lang, docs=docs, metadatas=metadatas)
    return sub_corpus

def fpx(year):
    return lambda x: int(x.metadata['signed_year']) == year

#data = corpus_subset(corpus, fpx(1947)).word_freqs(weighting='freq', as_strings=True)
#df = pd.DataFrame({'key': list(data.keys()),  'weight': list(data.values()) })
#df

In [ ]:
import itertools
from glove import Corpus
import pandas as pd
import scipy

# See http://www.foldl.me/2014/glove-python/
#def compute_GloVe_df(sentences, window=2, dictionary=None):
#    
#    corpus = Corpus(dictionary=dictionary)
#    corpus.fit(sentences, window=window)

#    dm = corpus.matrix.todense()
#    inverse_dictionary = { i: w for w, i in corpus.dictionary.items() }
#    id2token = [ inverse_dictionary[i] for i in range(0,max(inverse_dictionary.keys())+1)]

#    df = pd.DataFrame(dm.T, columns=id2token).assign(word=id2token).set_index('word')
#    return df

#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.nonzero.html
    
def _coo_to_sparse_series(A, dense_index=False):
    """ Convert a scipy.sparse.coo_matrix to a SparseSeries.
    Use the defaults given in the SparseSeries constructor. """
    # A = scipy.sparse.triu(A)
    s = pd.Series(A.data, pd.MultiIndex.from_arrays((A.row, A.col)))
    s = s.sort_index()
    s = s.to_sparse()
    return s

term_args = dict(
    args=dict(
        ngrams=1,
        named_entities=False,
        normalize='lemma',
        as_strings=True
    ),
    kwargs=dict(
        filter_stops=True,
        filter_punct=True,
        filter_nums=True,
        min_freq=1,
        drop_determiners=True,
        include_pos=('NOUN', 'PROPN', )
    )
)

stream = (textacy_filter_terms(doc, term_args) for doc in CORPUS)
#stream = (' '.join(list(textacy_filter_terms(doc, term_args))) for doc in CORPUS)

glove_co_matrix = Corpus() #dictionary=None)

docs = (list(textacy_filter_terms(doc, term_args)) for doc in CORPUS)
glove_co_matrix.fit(docs, window=5)

dictionary = glove_co_matrix.dictionary
co_series = _coo_to_sparse_series(glove_co_matrix.matrix)
co_df = co_series.to_dense().reset_index().rename(columns={ 'level_0': 'token_id_x', 'level_1': 'token_id_y', 0: 'weight'})
id2token = pd.DataFrame({ 'token_id': list(dictionary.values()), 'token': list(dictionary.keys()) }).set_index('token_id')

#https://stackoverflow.com/questions/34181494/populate-a-pandas-sparsedataframe-from-a-scipy-sparse-coo-matrix

df = pd.merge(co_df, id2token, left_on='token_id_x', right_index=True, how='inner').rename(columns={'token': 'token_x'})
df = pd.merge(df,    id2token, left_on='token_id_y', right_index=True, how='inner').rename(columns={'token': 'token_y'})


In [ ]:
#pd.DataFrame({'row': glove_co_matrix.matrix.row, 'col': glove_co_matrix.matrix.col}).groupby(['row', 'col']).size().nlargest()
#glove_co_matrix.matrix.tocoo()
#df.nlargest(100, ['weight'])
#df.groupby(['token_id_x', 'token_id_y']).size().nlargest()
#co_series.reset_index().groupby(['level_0', 'level_1']).size().nlargest()
dictionary


# TESTS AND WORK IN PROGRESS 

In [ ]:
corpus = get_current_corpus().textacy_corpus
term_args = {
    'mask_gpe': True,
    'kwargs': {'filter_stops': True, 'filter_punct': True, 'include_pos': ('NOUN', 'PROPN'), 'min_freq': 2},
    'extra_stop_words': set(),
    'args': {'normalize': 'lemma', 'named_entities': False, 'ngrams': [1], 'as_strings': True},
    'min_freq': 2,
    'max_doc_freq': 0.8
}
fx_terms = lambda: textacy_utility.extract_corpus_terms(corpus, term_args)

for x in fx_terms():
    print(len([y for y in x]))


In [ ]:
df = pd.DataFrame([ {
    'n_topics': int(model.tm_model.num_topics),
    'perplexity_score': model.perplexity_score,
    'coherence_score': model.coherence_score
  } for model in models ])
df['n_topics'] = df.n_topics.astype(int)
df = df.set_index('n_topics')
df['coherence_score'].plot.line()

### Create DataFrame Corpus from spaCy Documents

## Text Corpus vs WTI Index

In [ ]:

corpus_documents = corpus.documents.set_index(['treaty_id', 'language'])
treaty_text_languages = wti_index.get_treaty_text_languages().set_index(['treaty_id', 'language'])

treaties_in_corpus_not_in_wti = corpus_documents.index.difference(treaty_text_languages.index).get_values()
treaties_in_wti_not_in_corpus = treaty_text_languages.index.difference(corpus_documents.index).get_values()

print(  'Found in corpus, but not in WTI: ' +
        ', '.join([ '{}/{}'.format(x,y) for x,y in treaties_in_corpus_not_in_wti ]))

print(  'Found in WTI, but not in corpus: ' +
        ', '.join([ '{}/{}'.format(x,y) for x,y in treaties_in_wti_not_in_corpus ]))

#corpus_documents.loc[corpus_text_not_in_wti]
#treaty_text_languages.loc[wti_not_in_corpus]

#wti_not_in_corpus

# Duplicates:
#corpus_documents.index.get_duplicates()
#treaty_text_languages.corpus_documents.index.get_duplicates()


### Task: Basic Corpus Statistics
See https://www.nltk.org/book/ch01.html

* Size of treaties over time
* Unique word, unique words per word class
* Lexical diversity
* Frequency distribution
* Average word length, sentence length


```python
 
len(texts) / count(docs)
0.06230453042623537
len(set(text3)) / len(text3)
0.06230453042623537
def lexical_diversity(text): [1]
    return len(set(text)) / len(text) [2]

def percentage(count, total): [3]
    return 100 * count / total
#### Most common words
fdist1 = FreqDist(text1)
fdist1.most_common(50)
#### Word length frequencies
fdist = FreqDist(len(w) for w in text1)  [2]
print(fdist)
```

In [ ]:
# Code 

corpus = None
def display_token_toplist_interact(source_folder):
    global corpus
    progress_widget = None
    
    def display_token_toplist(source_folder, language, statistics='', remove_stopwords=False):
        global corpus

        try:

            progress_widget.value = 1

            corpus = TreatyCorpusSaveLoad(source_folder=source_folder, lang=language[0]).load_mm_corpus()

            progress_widget.value = 2
            service = MmCorpusStatisticsService(corpus, dictionary=corpus.dictionary, language=language)

            print("Corpus consists of {} documents, {} words in total and a vocabulary size of {} tokens."\
                      .format(len(corpus), corpus.dictionary.num_pos, len(corpus.dictionary)))

            progress_widget.value = 3
            if statistics == 'word_freqs':
                display(service.compute_word_frequencies(remove_stopwords))
            elif statistics == 'documents':
                display(service.compute_document_stats())
            elif statistics == 'word_count':
                display(service.compute_word_stats())
            else:
                print('Unknown: ' + statistics)

        except Exception as ex:
            logger.error(ex)

        progress_widget.value = 5
        progress_widget.value = 0
        return corpus
    
    language_widget=widgets.Dropdown(
        options={
            'English': ('en', 'english'),
            'French': ('fr', 'french'),
            'German': ('de', 'german'),
            'Italian': ('it', 'italian')
        },
        value=('en', 'english'),
        description='Language:', **dict(layout=widgets.Layout(width='260px'))
    )
    
    statistics_widget=widgets.Dropdown(
        options={
            'Word freqs': 'word_freqs',
            'Documents': 'documents',
            'Word count': 'word_count'
        },
        value='word_count',
        description='Statistics:', **dict(layout=widgets.Layout(width='260px'))
    )
    
    remove_stopwords_widget=widgets.ToggleButton(
        description='Remove stopwords', value=True,
        tooltip='Do not include stopwords in token toplist'
    )
    
    progress_widget=widgets.IntProgress(min=0, max=5, step=1, value=0) #, layout=widgets.Layout(width='100%')),

    wi = widgets.interactive(
        display_token_toplist,
        source_folder=source_folder,
        language=language_widget,
        statistics=statistics_widget,
        remove_stopwords=remove_stopwords_widget
    )

    boxes = widgets.HBox(
        [
            language_widget, statistics_widget, remove_stopwords_widget, progress_widget
        ]
    )
    display(widgets.VBox([boxes, wi.children[-1]]))
    wi.update()

display_token_toplist_interact('../data')

### <span style='color: red'>WORK IN PROGRESS</span> Task: Treaty Keyword Extraction (using TF-IDF weighing)
- [ML Wiki.org](http://mlwiki.org/index.php/TF-IDF)
- [Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
- Spärck Jones, K. (1972). "A Statistical Interpretation of Term Specificity and Its Application in Retrieval".
- Manning, C.D.; Raghavan, P.; Schutze, H. (2008). "Scoring, term weighting, and the vector space model". ([PDF](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf))
- https://markroxor.github.io/blog/tfidf-pivoted_norm/
$\frac{tf-idf}{\sqrt(rowSums( tf-idf^2 ) )}$
- https://nlp.stanford.edu/IR-book/html/htmledition/pivoted-normalized-document-length-1.html

Neural Network Methods in Natural Language Processing, Yoav Goldberg:
![image.png](attachment:image.png)

In [ ]:
# Code
from scipy.sparse import csr_matrix
%timeit

def get_top_tfidf_words(data, n_top=5):
    top_list = data.groupby(['treaty_id'])\
        .apply(lambda x: x.nlargest(n_top, 'score'))\
        .reset_index(level=0, drop=True)
    return top_list

def compute_tfidf_scores(corpus, dictionary, smartirs='ntc'):
    #model = gensim.models.logentropy_model.LogEntropyModel(corpus, normalize=True)
    model = gensim.models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary, normalize=True) #, smartirs=smartirs)
    rows, cols, scores = [], [], []
    for r, document in enumerate(corpus): 
        vector = model[document]
        c, v = zip(*vector)
        rows += (len(c) * [ int(r) ])
        cols += c
        scores += v
        
    return csr_matrix((scores, (rows, cols)))
    
if True: #'tfidf_cache' not in globals():
    tfidf_cache = {
    }
    
def display_tfidf_scores(source_folder, language, period, n_top=5, threshold=0.001):
    
    global state, tfw, tfidf_cache
    
    try:
        treaties = state.treaties

        tfw.progress.value = 0
        tfw.progress.value += 1
        if language[0] not in tfidf_cache.keys():
            corpus = TreatyCorpusSaveLoad(source_folder=source_folder, lang=language[0])\
                .load_mm_corpus(normalize_by_D=True)
            document_names = corpus.document_names
            dictionary = corpus.dictionary
            _ = dictionary[0]

            tfw.progress.value += 1
            A = compute_tfidf_scores(corpus, dictionary)

            tfw.progress.value += 1
            scores = pd.DataFrame(
                [ (i, j, dictionary.id2token[j], A[i, j]) for i, j in zip(*A.nonzero())],
                columns=['document_id', 'token_id', 'token', 'score']
            )
            tfw.progress.value += 1
            scores = scores.merge(document_names, how='inner', left_on='document_id', right_index=True)\
                .drop(['document_id', 'token_id', 'document_name'], axis=1)

            scores = scores[['treaty_id', 'token', 'score']]\
                .sort_values(['treaty_id', 'score'], ascending=[True, False])

            tfidf_cache[language[0]] = scores

        scores = tfidf_cache[language[0]]
        if threshold > 0:
            scores = scores.loc[scores.score >= threshold]

        tfw.progress.value += 1

        #scores = get_top_tfidf_words(scores, n_top=5)
        #scores = scores.groupby(['treaty_id']).sum() 

        scores = scores.groupby(['treaty_id'])\
            .apply(lambda x: x.nlargest(n_top, 'score'))\
            .reset_index(level=0, drop=True)\
            .set_index('treaty_id')

        if period is not None:
            periods = state.treaties[period]
            scores = scores.merge(periods.to_frame(), left_index=True, right_index=True, how='inner')\
                .groupby([period, 'token']).score.agg([np.mean])\
                .reset_index().rename(columns={0:'score'}) #.sort_values('token')

        #['token'].apply(' '.join)

        display(scores)
    except Exception as ex:
        logger.error(ex)
        
    tfw.progress.value = 0

#if 'tfidf_scores' not in globals():
#    tfidf_scores = compute_document_tfidf(corpus, corpus.dictionary, state.treaties)
#    tfidf_scores = tfidf_scores.sort_values(['treaty_id', 'score'], ascending=[True, False])

tfw = BaseWidgetUtility(
    language=widgets.Dropdown(
        options={
            'English': ('en', 'english'),
            'French': ('fr', 'french'),
            'German': ('de', 'german'),
            'Italian': ('it', 'italian')
        },
        value=('en', 'english'),
        description='Language:', **drop_style
    ),
    remove_stopwords=widgets.ToggleButton(
        description='Remove stopwords', value=True,
        tooltip='Do not include stopwords in token toplist', **toggle_style
    ),    
    n_top=widgets.IntSlider(
        value=5, min=1, max=25, step=1,
        description='Top #:',
        continuous_update=False
    ),
    threshold=widgets.FloatSlider(
        value=0.001, min=0.0, max=0.5, step=0.01,
        description='Threshold:',
        tooltip='Word having a TF-IDF score below this value is filtered out',
        continuous_update=False,
        readout_format='.3f',
    ), 
    period=widgets.Dropdown(
        options={
            '': None,
            'Year': 'signed_year',
            'Default division': 'signed_period',
            'Alt. division': 'signed_period_alt'
        },
        value='signed_period',
        description='Period:', **drop_style
    ),
    output=widgets.Dropdown(
        options={
            '': None,
            'Year': 'signed_year',
            'Default division': 'signed_period',
            'Alt. division': 'signed_period_alt'
        },
        value='signed_period',
        description='Output:', **drop_style
    ),
    progress=widgets.IntProgress(min=0, max=5, step=1, value=0) #, layout=widgets.Layout(width='100%')),
)

itfw = widgets.interactive(
    display_tfidf_scores,
    source_folder='./data',
    language=tfw.language,
    n_top=tfw.n_top,
    threshold=tfw.threshold,
    period=tfw.period
)

boxes = widgets.HBox(
    [
        widgets.VBox([tfw.language, tfw.period]),
        widgets.VBox([tfw.n_top, tfw.threshold]),
        widgets.VBox([tfw.progress, tfw.output])
    ]
)

display(widgets.VBox([boxes, itfw.children[-1]]))
itfw.update()


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from beakerx.object import beakerx
from beakerx import *
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

model = Cooccurrence(ngram_range=(1, 1))
Xc = model.fit_transform(corpus)

id2token = { i: x for (i, x) in enumerate(model.get_feature_names()) }

import pandas as pd

Xxy = Xc.tocoo()
word1 = [ id2token[x] for x in Xxy.row ]
word2 = [ id2token[x] for x in Xxy.col ]

df = pd.DataFrame({ 'word1': [ id2token[x] for x in Xxy.row ], 'word2': [ id2token[x] for x in Xxy.col ], 'count': Xxy.data })[['word1', 'word2', 'count']] #.set_index(['word1', 'word2'])
df

In [ ]:
def _coo_to_sparse_series(A, dense_index=False):
    """ Convert a scipy.sparse.coo_matrix to a SparseSeries.
    Use the defaults given in the SparseSeries constructor. """
    s = Series(A.data, MultiIndex.from_arrays((A.row, A.col)))
    s = s.sort_index()
    s = s.to_sparse()  # TODO: specify kind?
    # ...
    return s


In [ ]:
m